In [2]:
from igraph import *
import cairo
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

import csv
from itertools import groupby

import utils

%load_ext autoreload
%autoreload 2

In [21]:
data = pd.read_csv('processed/result.csv')
data.head()

,host,target
0,RNF170,TMA16
1,RNF170,SLC1A1
2,RNF170,CAPN15
3,RNF170,RCOR1
4,RNF170,PIK3C2B


In [6]:
data = pd.read_csv('processed/good.csv')
data.head()

,UNIT_ID,UnitName,EntrezID,GeneSymbol,GSM441627,GSM441628,GSM441629,GSM441643,GSM441644,GSM441657,...,GSM79360,GSM79353,GSM79153,GSM79349,GSM79355,GSM79181,GSM79323,GSM79214,GSM79350,GSM79211
0,25,AFFX-HUMISGF3A/M97935_MA_at,6772,STAT1,258.604,271.703,143.8760,96.2013,211.158,245.954,...,260.148,103.5360,174.966,191.4550,121.6530,122.4570,129.9380,178.902,251.261,103.5830
1,26,AFFX-HUMISGF3A/M97935_MB_at,6772,STAT1,114.433,148.662,81.2455,58.1097,135.255,169.862,...,214.424,45.7677,104.428,78.0305,58.6785,65.1835,76.1533,125.455,144.751,57.6514
2,27,AFFX-HUMISGF3A/M97935_3_at,6772,STAT1,609.059,562.069,349.9580,191.2970,879.009,771.731,...,1205.720,143.9720,789.292,1128.1100,287.0620,334.0280,386.8000,652.239,1164.150,269.3320
3,31,AFFX-HUMGAPDH/M33197_5_at,2597,GAPDH,4470.210,4570.150,2186.3100,4799.3800,6296.870,4864.770,...,2647.440,2678.1600,1562.820,1009.7900,4489.6300,2030.0600,2990.5000,2602.680,1699.050,2332.6800
4,32,AFFX-HUMGAPDH/M33197_M_at,2597,GAPDH,5183.690,4677.370,2771.9800,5587.2800,7326.030,5456.950,...,3957.630,4781.1200,2632.850,2252.8800,6344.9700,3244.9100,4166.8100,3619.780,2916.770,4011.9500


In [7]:
data = pd.read_csv('processed/total.csv')
data.head()

,GSM441627,TooLate0
0,GSM441628,relapse
1,GSM441629,relapse
2,GSM441643,relapse
3,GSM441644,relapse
4,GSM441657,relapse


In [25]:
g = utils.set_graph(data, True)

In [ ]:
knet = utils.get_knet(g, 1)
print(knet)

In [29]:
g.diameter()

9

In [15]:
g.degree()[:5]

[328, 114, 1301, 102, 447]

## Top 3 by degree

In [16]:
topList = sorted([[ind, val] for ind, val in enumerate(g.degree())], key=lambda x: x[1], reverse=True)[:3]
list(map(lambda node: g.vs[node[0]]['name'], topList))

['MEST', 'MCM7', 'C9orf3']

## Top 3 by IN degree

In [17]:
topList = sorted([[ind, val] for ind, val in enumerate(g.degree(mode="in"))], key=lambda x: x[1], reverse=True)[:3]
list(map(lambda node: g.vs[node[0]]['name'], topList))

['CDKN1A', 'ZNF460', 'AGO2']

## Top 3 by OUT degree

In [18]:
topList = sorted([[ind, val] for ind, val in enumerate(g.degree(mode="out"))], key=lambda x: x[1], reverse=True)[:3]
list(map(lambda node: g.vs[node[0]]['name'], topList))

['MEST', 'MCM7', 'C9orf3']

In [19]:
summary(g.components().cluster_graph())

IGRAPH D--- 13514 62702 -- 


In [10]:
# get strong connected components
components = g.components()
subgraphs = components.subgraphs()

# vertex counts for subgraphs
ordered_counts = sorted([graph.vcount() for graph in subgraphs])
x = [list(group)[0] for key, group in groupby(ordered_counts)]
y = [len(list(group)) for key, group in groupby(ordered_counts)]

In [11]:
print(x)
print(y)

[1, 674]
[13513, 1]


In [12]:
max([graph.diameter() for graph in subgraphs])

8